In [1]:
import glob
import os, sys
os.path.abspath(os.getcwd())
sys.path.append("C:\\Users\\abel_\\Documents\\Rotations\\CIT\\cit_for_computation")

import torch
import torch.nn as nn
import pandas
import numpy as np
import numpy.ma as ma
import pickle
import time
from tqdm.notebook import tqdm
import matplotlib.cm as cmx
import sklearn
import sklearn.decomposition

import proplot as pplt
from fractions import Fraction

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from utils import *
from tasks import DynamicPoissonClicks

C:\Users\abel_\anaconda3\envs\CIT_\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
current_dir = 'C:\\Users\\abel_\\Documents\\Rotations\\CIT\cit_for_computation'
exp_extension = "\\experiments\\dynamic_poisson_clicks\\"

exp_list = glob.glob(current_dir + exp_extension + "\\*")
exp_path = exp_list[1]
with open(exp_path + '\\training_kwargs.pickle', 'rb') as handle:
    training_kwargs = pickle.load(handle)

training_kwargs['exp_path'] = exp_path
training_kwargs['training_weights_path'] = exp_path + '/training/'
training_kwargs['figures_path'] = exp_path + '//figures/'

N_rec = training_kwargs['N_rec']
N_in =  training_kwargs['N_in']
N_out = training_kwargs['N_out']

rnn_model = RNNModel(N_in, N_out, N_rec, n_layers=1, transform_function='relu').to(device)
rnn_model = load_model(rnn_model, training_kwargs).to(device)
exp_path

'C:\\Users\\abel_\\Documents\\Rotations\\CIT\\cit_for_computation\\experiments\\dynamic_poisson_clicks\\exp_bce_cosineannealing_50_rnn_cued_Nrec5_1666082116'

In [3]:
#load
with open(exp_path + "\\dataset.pickle", 'rb') as handle:
    dataset = pickle.load(handle)
train_set, test_set = dataset
test_x, test_y, output_mask, trial_params = train_set
    
# #create
# pdc = DynamicPoissonClicks(10000, training_kwargs)
# test_x, test_y, output_mask, trial_params =  pdc.get_trial_batch()

inputs = torch.tensor(test_x, dtype=torch.float)
yhat, hidden = rnn_model(inputs)
yhat = yhat.cpu().detach().numpy()

trial_params = np.array(trial_params)

In [11]:
chosen = np.argmax(np.mean(yhat*output_mask, axis=1), axis = 1)
truth = np.argmax(np.mean(test_y*output_mask, axis = 1), axis = 1)
response_correctness = np.equal(truth, chosen)
error_idx = np.where(response_correctness==False)[0]
correct_idx = np.where(response_correctness==True)[0]

accuracy = pd_accuracy_function(test_y, yhat, output_mask)
print(accuracy)

0.997625


In [12]:
from scipy.optimize import minimize
# f = lambda x: 

# θ can be described in words as an initial noise variance σi2, a per-click noise variance σs2,
# a memory noise variance \sigma_a^2,
# a discounting rate λ,
# the strength and time constant of adaptation φ and τ_φ,
# a decision boundary B,
# which captures the animal’s bias,
# and a lapse rate l.

In [13]:
#eq 10 from Boyd-Meredith
# each moment in the trial, the forward model f(a) = P(t, δR, δL, θ, a_0 \sim N(0; \sigma_i^2)) predicts a Gaussian distribution of accumulation values with mean \mu(t) and variance \sigma^2(t) given by:
def mu_t(params, t, clicks):
    "clicks includes the clicks times for left (0, i.e. first column) and right (1, second col)"
    l, lam, sigma_init, sigma_s, simga_a, B = params[:6]
    C = params[6:]
    return np.sum(np.exp(lam*(t-clicks[1].astype(float)))*C[clicks[1].astype(int)]) - np.sum(np.exp(lam*(t-clicks[0].astype(float)))*C[clicks[0].astype(int)])

In [14]:
trial_i = 0
clicks= np.array([np.where(test_x[trial_i,:,0]>0)[0], np.where(test_x[trial_i,:,1]>0)[0]], dtype=list)
t = 10
lam = 1.
C = 0.5*np.ones((2000))
sigma_init = 0.1
sigma_s = 1
sigma_a = 1
B = 1
l = 0.1
params = np.concatenate(([l, lam, sigma_init, sigma_s, sigma_a, B], C))

-1.8126726864874158e-07

In [15]:
#eq 11: variance
def sigma_t(params, t, clicks):
    l, lam, sigma_init, sigma_s, simga_a, B = params[:6]
    C = params[6:]
    integral = sigma_s**2*(np.sum(np.exp(2*lam*(t-clicks[1].astype(float)))*C[clicks[1].astype(int)]) - np.sum(np.exp(2*lam*(t-clicks[0].astype(float)))*C[clicks[0].astype(int)]))
    return sigma_init**2*np.exp(2*lam*t) + sigma_a**2*(np.exp(2*lam*t)-1)/(2*lam) + integral

In [16]:
#eq 12: To determine the probability of a right versus left choice, we first integrate the accumulation value distribution in the last timepoint tN of the trial from the decision boundary parameter B to ∞

def prob_of_rvsl(params, t_N, clicks):
    mu_tN = mu_t(params, t_N, clicks)
    sigma_tN = sigma_t(params, t, clicks)
    return 0.5*(1+scipy.special.erf(-(B-mu_tN)/(sigma_tN*np.sqrt(2))))

In [17]:
#eq 13/14
def prob_goright(params, y, t_N, clicks):
    l = params[-1]
    prob_of_r = prob_of_rvsl(params, t_N, clicks)
    if y==1.:
        return (1-l)*prob_of_r+l/2.
    elif y==-1.:
        return (1-l)*(1-prob_of_r)+l/2.

In [18]:
trial_i = 0
t_N = np.argmax(test_y[trial_i,:,0]!=0.5)-1 #-1 or not?
y = 2*(chosen[trial_i]-.5)

In [19]:
n_trials = test_x.shape[0]
all_clicks = [np.array([np.where(test_x[trial_i,:,0]>0)[0], np.where(test_x[trial_i,:,1]>0)[0]], dtype=list) for trial_i in range(n_trials)]
# all_clicks = [np.array([np.where(test_x[trial_i,:,0]>0)[0].astype(np.float), np.where(test_x[trial_i,:,1]>0)[0].astype(np.float)], dtype=list) for trial_i in range(n_trials)]

t_Ns = [np.argmax(test_y[trial_i,:,0]!=0.5)-1 for trial_i in range(n_trials)]
ys = [2*(chosen[trial_i]-.5) for trial_i in range(n_trials)]

In [20]:
# likelihood = prod_i prob_goright()[i]
def log_likelihood_all(params, ys, t_Ns, all_clicks):
    "log likelihood of data given parameters"
    log_likelihoods = []
    for trial_i in range(len(all_clicks)):
        clicks = all_clicks[trial_i]
        t_N = t_Ns[trial_i]
        y = ys[trial_i]
        log_likelihood_i = np.log(prob_goright(params, y, t_N, clicks))
        log_likelihoods.append(log_likelihood_i)
    return -np.sum(log_likelihoods)

In [22]:
x0 = np.array(params)

In [23]:
res = minimize(log_likelihood_all, x0=x0, args=(ys, t_Ns, all_clicks), tol=1e-6)